# **Ridge 와 Lasso**

# **다중회귀**
- 여러개의 독립변수
- ax + by + cz + d
- **특성공학** = 각 특성을 곱해서 또 다른 특성을 만드는 것

In [41]:
import pandas as pd
df = pd.read_csv('data/Fish.csv')
perch_df = df.loc[df['Species']=='Perch']
perch_df.head()

,Species,Weight,Length1,Length2,Length3,Height,Width
72,Perch,5.9,7.5,8.4,8.8,2.1120,1.4080
73,Perch,32.0,12.5,13.7,14.7,3.5280,1.9992
74,Perch,40.0,13.8,15.0,16.0,3.8240,2.4320
75,Perch,51.5,15.0,16.2,17.2,4.5924,2.6316
76,Perch,70.0,15.7,17.4,18.5,4.5880,2.9415


In [42]:
# Length2, HEight, Width 사용
perch_full = perch_df[['Length2','Height','Width']]
perch_weight = perch_df[['Weight']]

### 훈련 데이터와 test 데이터 생성하기

In [43]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(perch_full, perch_weight)

### 데이터 전처리 (사이킷런 변환기)
- 각 특성을 제곱한 항과 특성끼리 곱한 항을 추가한다

In [44]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(include_bias = False) # False로하면 1 이 빠짐
poly.fit([[3,5]])
poly.transform([[3,5]])

array([[ 3.,  5.,  9., 15., 25.]])

### PolynomialFeatures- train_input 
- 계수간 상호작용을 통해 새로운 Feature를 생성

In [45]:
poly = PolynomialFeatures(include_bias = False)
poly.fit(train_input)  #역할은 없지만 항상 fit을 해야함
train_poly = poly.transform(train_input)
train_poly.shape # (42, 9) => 새로운 특성 9개가 만들어짐

(42, 9)

In [46]:
poly.get_feature_names_out()

array(['Length2', 'Height', 'Width', 'Length2^2', 'Length2 Height',
       'Length2 Width', 'Height^2', 'Height Width', 'Width^2'],
      dtype=object)

In [47]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression() # 변수에 인스턴스화
lr.fit(train_poly, train_target)
lr.score(train_poly, train_target)

0.9937693677822032

### PolynomialFeatures- test_input 

In [48]:
test_poly = poly.transform(test_input)
lr.score(test_poly, test_target)

0.9578406036686441

In [49]:
#a * 길이 + b * 높이 + c * 너비 + d * 길이^2 + e * 높이 * 길이

In [50]:
poly = PolynomialFeatures(degree=5, include_bias = False)
poly.fit(train_input)
train_poly = poly.transform(train_input)
test_poly = poly.transform(test_input)
train_poly.shape  #(42,55) => degree=5일때 55개의 특성이 생성됨

(42, 55)

In [51]:
lr.fit(train_poly, train_target)

LinearRegression()

In [52]:
print(lr.score(train_poly, train_target))
print(lr.score(test_poly, test_target))

0.9999999999999912
-13.00136177053976


# **규제**
- 머신러닝 모델이 훈련세트에 과대적합되지 않도록 만드는 것
- 계수를 작게 만든다

In [53]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
ss.fit(train_poly)
train_scaled = ss.transform(train_poly)
test_scaled = ss.transform(test_poly)


# **Ridge 회귀** 

In [56]:
from sklearn.linear_model import Ridge
ridge = Ridge(alpha=0.1)
ridge.fit(train_scaled, train_target)

print(ridge.score(train_scaled, train_target))
print(ridge.score(test_scaled, test_target))

0.9939849018538158
0.9755916005048859


# **Lasso**
- Ridge와 달리 계수를 0으로 만들수 있다

In [59]:
from sklearn.linear_model import Lasso
lasso = Lasso()
lasso.fit(train_scaled, train_target)

print(lasso.score(train_scaled, train_target))
print(lasso.score(test_scaled, test_target))

0.9886619716657188
0.9845761696649022


C:\Users\1-15\Desktop\DAMF2\ML\venv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.433e+02, tolerance: 4.779e+02
  model = cd_fast.enet_coordinate_descent(


In [60]:
lasso.coef_

array([  0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,  27.62730479,   0.        ,
        57.72929337, 109.11628813,  35.2077218 ,   0.        ,
        48.16701427,   0.        ,   0.        ,  59.64157653,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
        -0.        ,   0.        ,  -0.        ,  -0.        ,
         0.        ,   0.        ,  -0.        ,  -0.        ,
         0.        ,   0.        ,  -0.        ,  -0.        ,
        -0.        ,   0.        ,   0.        ,  -0.        ,
        -0.        ,  -0.        ,  -0.        ])